<a href="https://colab.research.google.com/github/Abhisheklal2809/Algorithm--ACO-ORS/blob/main/TSP_ACO_OpenRoute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openrouteservice folium

In [3]:
import openrouteservice
import numpy as np
import time
import folium
from google.colab import files
import random

In [4]:
# ORS API Key
API_KEY = "5b3ce3597851110001cf6248ecec7d0fd7704fd8b1b5d83331296e14"
client = openrouteservice.Client(key=API_KEY)

In [5]:
# Get coordinates from location names
def get_coordinates(locations):
    coords = []
    for loc in locations:
        try:
            response = client.pelias_search(loc)
            if response and 'features' in response and response['features']:
                coord = response['features'][0]['geometry']['coordinates'][::-1]
                coords.append(tuple(coord))
                time.sleep(1)
            else:
                print(f" No coordinates found for {loc}")
                return None
        except Exception as e:
            print(f" Error fetching coordinates for {loc}: {e}")
            return None
    return coords


In [6]:
# Build real-time travel time matrix
def get_distance_matrix(coords):
    try:
        n = len(coords)
        matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    response = client.directions(
                        coordinates=[coords[i][::-1], coords[j][::-1]],
                        profile="driving-car",
                        format="json"
                    )
                    if "routes" in response and response["routes"]:
                        matrix[i][j] = response["routes"][0]["summary"]["duration"]
                        time.sleep(1)  # Avoid rate-limiting
                    else:
                        print(f" No route between {coords[i]} and {coords[j]}")
                        return None
        return matrix
    except Exception as e:
        print(f" Error creating distance matrix: {e}")
        return None

In [7]:
# Ant Colony Optimization (starts at city[0], no return)
def solve_tsp_aco(matrix, num_ants=10, num_iterations=100, alpha=1.0, beta=2.0, evaporation_rate=0.5, Q=100):
    n = len(matrix)
    pheromone = np.ones((n, n))
    best_path = None
    best_cost = float('inf')

    def path_cost(path):
        return sum(matrix[path[i]][path[i + 1]] for i in range(len(path) - 1))

    for iteration in range(num_iterations):
        all_paths = []
        all_costs = []

        for _ in range(num_ants):
            unvisited = list(range(1, n))  # start fixed at city 0
            path = [0]

            while unvisited:
                current = path[-1]
                probabilities = []
                for next_city in unvisited:
                    pher = pheromone[current][next_city] ** alpha
                    heur = (1 / matrix[current][next_city]) ** beta
                    probabilities.append(pher * heur)

                total = sum(probabilities)
                probabilities = [p / total for p in probabilities]
                next_city = random.choices(unvisited, weights=probabilities)[0]
                path.append(next_city)
                unvisited.remove(next_city)

            all_paths.append(path)
            cost = path_cost(path)
            all_costs.append(cost)

            if cost < best_cost:
                best_cost = cost
                best_path = path

        # Update pheromones
        pheromone *= (1 - evaporation_rate)
        for path, cost in zip(all_paths, all_costs):
            for i in range(len(path) - 1):
                a, b = path[i], path[i + 1]
                pheromone[a][b] += Q / cost
                pheromone[b][a] += Q / cost

    return best_path, best_cost

In [8]:
# Format city path
def format_route(locations, path):
    return [locations[i] for i in path]

# Step 5: Plot the path using Folium (includes return leg)
def plot_route(coords, path, locations):
    m = folium.Map(location=coords[0], zoom_start=6)

    for i, coord in enumerate(coords):
        folium.Marker(coord, popup=locations[i], icon=folium.Icon(color='blue')).add_to(m)

    route_coords = [coords[i] for i in path]
    folium.PolyLine(route_coords, color='blue', weight=3, opacity=1, tooltip="ACO Path").add_to(m)

    # Highlight start and end
    folium.Marker(coords[path[0]], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[path[-1]], popup="End", icon=folium.Icon(color='red')).add_to(m)

    # Draw return leg as dashed red line
    folium.PolyLine(
        [coords[path[-1]], coords[path[0]]],
        color='red',
        weight=2,
        opacity=0.7,
        dash_array='5,10',
        tooltip='Return to Start'
    ).add_to(m)

    return m

In [ ]:
#  MAIN
locations = [
    "Delhi, India", "Agra, India", "Jaipur, India", "Lucknow, India", "Chandigarh, India", "Allahabad, India", "Bhagalpur, India"
]

print(" Fetching coordinates...")
coords = get_coordinates(locations)

if coords:
    print(" Creating distance matrix...")
    matrix = get_distance_matrix(coords)

    print(" Running Ant Colony Optimization (No return to start)...")
    best_path, best_cost = solve_tsp_aco(matrix)

    print("\n Optimal Route (Start at Delhi, No Return):")
    print(format_route(locations, best_path))
    print(f" Travel Time (Path Only): {best_cost / 60:.2f} minutes")

    #  Add return time calculation
    start_coord = coords[best_path[0]][::-1]
    end_coord = coords[best_path[-1]][::-1]

    try:
        return_route = client.directions(
            coordinates=[end_coord, start_coord],
            profile="driving-car",
            format="json"
        )
        return_time = return_route["routes"][0]["summary"]["duration"]
        total_time = best_cost + return_time
        print(f" Return Time (End ➝ Start): {return_time / 60:.2f} minutes")
        print(f" Total Travel Time (Path + Return): {best_cost / 60:.2f} + {return_time / 60:.2f} = {total_time / 60:.2f} minutes")
    except Exception as e:
        print(f"Could not fetch return time: {e}")
        return_time = None

    #  Plot map and show return
    map_result = plot_route(coords, best_path, locations)
    map_file = "/content/aco_tsp_with_return.html"
    map_result.save(map_file)
    files.download(map_file)
    print(" Route map saved and downloaded.")
else:
    print(" Failed to get coordinates.")

 Fetching coordinates...
 Creating distance matrix...


/usr/local/lib/python3.12/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.12/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.12/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.12/dist-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/usr/local/lib/python3.12/dist-packages/openrouteservice/client.py:211: UserWarning: Rat